
**Equipo VIABA: Despliegue y Monitorización de Flujos de Prefect en Kubernetes**

Este tutorial guía a cada miembro del equipo a través de la configuración de un entorno de desarrollo, la creación de un flujo de Prefect, la definición de un deployment usando `flow.deploy()`, la configuración de un work pool de Kubernetes, el despliegue del flujo en Kubernetes, *y la autenticación correcta en Prefect Cloud*.

**Secciones:**

1.  Preparación del Entorno Ubuntu (Nativo o WSL2)
2.  Instalación de Docker
3.  Instalación de Minikube 
4.  Configuración del Entorno Virtual de Python y las Dependencias
5.  Configuración de Prefect Cloud 
6.  Creación y Subida de la Imagen de Docker (Dockerfile)
7.  Configuración del Work Pool de Kubernetes en Prefect Cloud
8.  Definición del Flujo y el Deployment con `flow.deploy()`
9.  Despliegue y Monitorización del Flujo

**1. Preparación del Entorno Ubuntu (Nativo o WSL2)**

1.  **Si usas WSL2:**

    *   Abre PowerShell como administrador.
    *   Ejecuta:

        ```powershell
        wsl --set-default-version 2
        ```

        *Abre la terminal de Ubuntu y continua con los demás comandos.*

    Si tienes Ubuntu nativo, no es necesario

2.  **Actualizar Ubuntu:**

    ```bash
    sudo apt update && sudo apt upgrade -y
    ```

**2. Instalación de Docker**

1.  **Instalar Docker:**

    *   Los pasos principales son:

        ```bash
        sudo apt update
        sudo apt install apt-transport-https ca-certificates curl gnupg
        curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
        echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
        sudo apt update
        sudo apt install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
        ```

    *   **Añadir tu usuario al grupo `docker`:**

        ```bash
        sudo groupadd docker  # Si no existe
        sudo usermod -aG docker $USER
        newgrp docker
        ```

        *Cierra y vuelve a abrir tu terminal después de ejecutar*

2.  **Configurar Docker para WSL2 (Si Usas WSL2):**

    *   Abre Docker Desktop (en Windows).
    *   Ve a "Settings" -> "Resources" -> "WSL INTEGRATION".
    *   Asegúrate de que la distribución de Ubuntu que instalaste esté seleccionada.

**3. Instalación de Minikube (O Acceso a un Clúster de Kubernetes)**

*   Instala Minikube:

    ```bash
    curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
    sudo install minikube-linux-amd64 /usr/local/bin/minikube
    minikube start --driver=docker --kubernetes-version=v1.27.3 --container-runtime=containerd
    ```

    *Verificar que esa versión sea la última estable*

*   Asegúrate de que `kubectl` está instalado para monitorizar los pods.

**4. Configuración del Entorno Virtual de Python y las Dependencias**

1.  **Verificar la Instalación de Python 3.12:**

    ```bash
    python3 --version
    ```

2.  **Crear un Entorno Virtual:**

    ```bash
    python3 -m venv viaba
    ```

3.  **Activar el Entorno Virtual:**

    ```bash
    source viaba/bin/activate
    ```

4.  **Instalar Dependencias (Dentro del Entorno Virtual):**

    ```bash
    pip3 install prefect mlflow dagshub prefect-kubernetes prefect-github psutil opencv-python scikit-learn
    ```

**5. Configuración de Prefect Cloud**

1.  **Obtener tu Clave de API y el Workspace:**

    El link esta en el grupo de whatsapp para iniciar directamente el prefect cloud(no se puede monitorizar todos pero crearemos una cuenta de grupo)

2.  **Configurar las Variables de Entorno:**

    *Asegúrate de configurar estas variables en tu archivo `~/.bashrc` y de que estén disponibles en tu entorno virtual* para que se carguen automáticamente al iniciar la terminal:*

    ```bash
    export PREFECT_API_KEY="pnu_BOXTYt4bnQc2t3j8QTQkgZHcAKokbM27UlMI"
    export PREFECT_API_URL="https://api.prefect.cloud/api/accounts/abc-my-account-id-is-here/workspaces/420db07f-97af-45a6-9a5f-087dbedce23f" # Acabaremos usando una cuenta global
    export DAGSHUB_TOKEN="tu_token_de_dagshub"  # Token personal de dagshub
    export DAGSHUB_USERNAME="tu_usuario_de_dagshub"  
    export DAGSHUB_REPO_NAME="-VIABA-VigilancIA-Computer-Vision-machine-learning" 
    export MLFLOW_TRACKING_URI="https://dagshub.com/reitz0770/-VIABA-VigilancIA-Computer-Vision-machine-learning.mlflow"  # URL de seguimiento de MLflow
    ```

**6. Creación y Subida de la Imagen de Docker ()**

1.  **Estructura del Directorio:**

    Para crear una imagen de Docker necesitamos una carpeta con:
    ```
    mi-proyecto/
    ├── train_flow.py  # Archivo de flujo de Prefect
    ├── requirements.txt  # Archivo con las dependencias
    └── Dockerfile        # El archivo de definición de la imagen de Docker
    ```

2.  **Crear un Dockerfile:**

    ```dockerfile
    FROM python:3.9-slim-buster

    WORKDIR /app

    COPY train_flow.py .
    COPY requirements.txt .

    RUN pip install --no-cache-dir -r requirements.txt

    CMD ["python", "train_flow.py"]
    ```

3.  **Crear un `requirements.txt`**

    ```text
    prefect
    mlflow
    dagshub
    prefect-kubernetes
    prefect-github
    psutil
    opencv-python
    scikit-learn
    ```

4.  **Build del dockerFile, para confirmar que está bueno**:

    *Si hay errores de permisos, prueba con: sudo chmod 777 train_flow.py && sudo chmod 777 requirements.txt*

    ```bash
    docker build -t reitzz/viaba-training-flow:latest .
    ```

    Verificar que las dependencias y dependencias del sistema del engine del comando anterior.

5.  Ahora debes loguearte en el repositorio de dockerHub

    ```bash
    docker login
    ```

6.  Con el login ya seteado, ejecutar el push con:

    ```bash
    docker push reitzz/viaba-training-flow:latest
    ```

    *Recuerda usar el valor de la configuración ya seteada, Reitz ya te la ha dado, recuerda esto, y que cada uno le ponga el nombre que quiere.*

**7. Configuración del Work Pool de Kubernetes en Prefect Cloud**

1.  **Crear el Work Pool en la UI de Prefect Cloud:**

    *   Ve a tu panel de control de Prefect Cloud.
    *   Navega a la sección "Work Pools".
    *   Haz clic en "Create Work Pool".
    *   Dale un nombre a tu work pool (por ejemplo, "kubernetes-viaba").
    *   Selecciona "Kubernetes" como el tipo de work pool.
    *   Configura opciones adicionales según las necesidades (límites de concurrencia, etc.).

**8. Definición del Flujo y el Deployment con `flow.deploy()`**

1.  **Crear el Archivo `train_flow.py` :**


    ```python
    from prefect import flow
    import os

    @flow(log_prints=True)
    def train_model(data_path: str, model_name: str = "my_model"):
        print(f"Entrenando modelo {model_name} con datos de {data_path}")
        #Entrenamiento llamando a tasks
        print("¡Modelo entrenado!")
        return model_name

    if __name__ == "__main__":
        train_model.deploy(
            name="viaba-training-deployment",
            work_pool_name="kubernetes-viaba",  # ¡Nombre del workpool que vayas a utilizar o que hayas creado
            image="reitzz/viaba-training-flow:latest",  # Imagen de docker, si no la encuentra y push=true, te hara un dockerfile automáticamente! Esto es lo bueno del prefect
            push=False,  
            parameters={"data_path": "/data/training_data.csv"},
            job_variables={"env": {"MLFLOW_TRACKING_URI": os.environ.get("MLFLOW_TRACKING_URI")}}
        )
    ```

2.  **Ejecutar el Archivo Python:**

    ```bash
    python train_flow.py
    ```

    *   Prefect registrará el deployment en Prefect Cloud.
    *   Si `push=True`, intentará subir la imagen de Docker a un registro.

**9. Despliegue y Monitorización del Flujo**

1.  **Iniciar un Worker de Prefect:**

    Necesitas un worker conectado a tu work pool de Kubernetes para recoger y ejecutar los flow runs.

    ```bash
    prefect worker start --pool "kubernetes-viaba"
    ```

    Reemplaza `"kubernetes-viaba"` con el nombre de la workpool que hayas designado en el train_flow.py, si ya existe lo usa, si no lo crea, este

2.  **Monitorizar en Prefect Cloud:**

    *   Ve a la UI de Prefect Cloud.
    *   Sección "Deployments".
    *   Deberías ver tu deployment y lanzar runs desde este.
    *   Cuando se ejecute un flow run si tienes la work pool activa, podrás ver su estado, logs y resultados en la UI.
